# Import Library and Load Data

In [1]:
from tqdm.auto import tqdm
import torch, torchtext
from torch import nn
import torch.nn.functional as F
import random, math, time
from datasets import load_dataset
import pandas as pd
import numpy as np
import re

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
# device = torch.device("cpu")
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cuda:1


In [2]:
import torch, torchtext, transformers
import numpy
print(torch.__version__, torchtext.__version__, transformers.__version__)
print(numpy.__version__)


2.2.0+cu121 0.16.2+cpu 4.39.3
1.26.4


In [5]:
from datasets import load_from_disk

# Load the dataset from disk
loaded_dataset = load_from_disk('dataset_edit')


In [6]:
dataset = loaded_dataset

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text1', 'entity', 'label'],
        num_rows: 452057
    })
    validation: Dataset({
        features: ['text1', 'entity', 'label'],
        num_rows: 83926
    })
    test: Dataset({
        features: ['text1', 'entity', 'label'],
        num_rows: 153206
    })
})

In [8]:
dataset["train"] = dataset["train"].rename_column("label", "labels")
dataset["validation"] = dataset["validation"].rename_column("label", "labels")
dataset["test"] = dataset["test"].rename_column("label", "labels")

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text1', 'entity', 'labels'],
        num_rows: 452057
    })
    validation: Dataset({
        features: ['text1', 'entity', 'labels'],
        num_rows: 83926
    })
    test: Dataset({
        features: ['text1', 'entity', 'labels'],
        num_rows: 153206
    })
})

In [10]:
from classModel import ContextBertModel, QACGBertForSequenceClassification

# Tokenization

In [21]:
# from datasets import load_dataset
# from transformers import BertTokenizer

# # Load the tokenizer
# tokenizer = BertTokenizer.from_pretrained('ibm-research/CTI-BERT')

# # Function to tokenize the text data
# def tokenize_function(examples):
#     return tokenizer(examples['text1'], padding="max_length", truncation=True, max_length=64)

# # Apply tokenization to train and validation datasets
# train_data = dataset['train'].map(tokenize_function, batched=True)
# val_data = dataset['validation'].map(tokenize_function, batched=True)
# test_data = dataset['test'].map(tokenize_function, batched=True)

# # Print an example to verify
# print(train_data[0]['text1'])  # It should show tokenized input

# # # Decode the tokenized text back to human-readable text
# # decoded_text = tokenizer.decode(train_data[0]['input_ids'], skip_special_tokens=True)
# # print(f"Decoded Text: {decoded_text}")


In [22]:
# # Save each split to disk
# train_data.save_to_disk("tokenized_data_edit/train")
# val_data.save_to_disk("tokenized_data_edit/validation")
# test_data.save_to_disk("tokenized_data_edit/test")


In [11]:
from datasets import load_from_disk
# .select(range(10000))
train_data = load_from_disk("tokenized_data_edit/train")
val_data = load_from_disk("tokenized_data_edit/validation")
test_data = load_from_disk("tokenized_data_edit/test")


In [12]:
train_data = train_data.rename_column("label", "labels")
val_data = val_data.rename_column("label", "labels")
test_data = test_data.rename_column("label", "labels")

In [13]:
train_data

Dataset({
    features: ['text1', 'entity', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 452057
})

# Data Processing

In [14]:
# Extract unique labels (MITRE techniques) from both train and validation datasets
labels = list(set(dataset['train']['labels']).union(set(dataset['validation']['labels'])))  # Extract unique labels
label_map = {label: i for i, label in enumerate(labels)}

# Function to encode labels into integers
def encode_labels(examples):
    # Safely map the labels, providing a default value if a label is not found in the label_map
    examples['labels'] = [label_map.get(label, -1) for label in examples['labels']]
    
    return examples

# Apply label encoding
train_data = train_data.map(encode_labels, batched=True)
val_data = val_data.map(encode_labels, batched=True)

# Print an example to verify
print(train_data[3])  # It should show tokenized input along with the encoded label


Map:   0%|          | 0/452057 [00:00<?, ? examples/s]

Map:   0%|          | 0/83926 [00:00<?, ? examples/s]

{'text1': 'These Microsoft Office templates are hosted on a command and control server and the downloaded link is embedded in the first stage malicious document', 'entity': 'I-TOOL', 'labels': 290, 'input_ids': [2, 455, 755, 1899, 8818, 226, 4700, 158, 43, 979, 137, 656, 629, 137, 114, 3633, 695, 146, 3718, 120, 114, 773, 3872, 815, 1355, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [15]:
print(dataset['train'][5]['labels'])

T1113


In [16]:
# Make sure each 'entity' is treated as a full string, not split into characters
unique_aspects = list(set(aspects for aspects in dataset['train']['entity']))
context_map = {asp: i+1 for i, asp in enumerate(unique_aspects)}  # +1 to reserve 0 for padding

# Update encoding accordingly
def encode_context(examples):
    examples["context_ids"] = [
        [context_map[a] for a in aspects if a in context_map] if isinstance(aspects, list) else [context_map.get(aspects, 0)]
        for aspects in examples["entity"]
    ]
    return examples

# Apply context encoding
train_data = train_data.map(encode_context, batched=True)
val_data = val_data.map(encode_context, batched=True)

# Print an example to verify
print(train_data[3])  # It should show tokenized input along with the encoded label

Map:   0%|          | 0/452057 [00:00<?, ? examples/s]

Map:   0%|          | 0/83926 [00:00<?, ? examples/s]

{'text1': 'These Microsoft Office templates are hosted on a command and control server and the downloaded link is embedded in the first stage malicious document', 'entity': 'I-TOOL', 'labels': 290, 'input_ids': [2, 455, 755, 1899, 8818, 226, 4700, 158, 43, 979, 137, 656, 629, 137, 114, 3633, 695, 146, 3718, 120, 114, 773, 3872, 815, 1355, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'context_ids': [15]}


In [31]:
# max_id = max(max(x["context_ids"]) for x in train_data)
# print("🔍 Max context ID:", max_id)


In [32]:
# max_context_id = 0
# for i in range(len(train_data)):
#     ids = train_data[i]["context_ids"]
#     max_context_id = max(max_context_id, max(ids))

# print("🚨 Max context_id in dataset:", max_context_id)


In [33]:
# print("✅ context_embeddings size:", model.bert.context_embeddings.num_embeddings)


In [17]:
import torch
from torch.utils.data import DataLoader, Dataset

class TTPDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, label_key='labels', max_context_id=None):
        self.dataset = hf_dataset
        self.label_key = label_key
        self.max_context_id = max_context_id

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        context_ids = torch.tensor(item["context_ids"], dtype=torch.long)

        if self.max_context_id is not None:
            context_ids = torch.clamp(context_ids, min=0, max=self.max_context_id)

        label = item[self.label_key]
        if label == -1:
            # Replace -1 with a default class, like 0
            label = 0  # or skip the example entirely


        return {
            "input_ids": torch.tensor(item["input_ids"]),
            "token_type_ids": torch.tensor(item["token_type_ids"]),
            "attention_mask": torch.tensor(item["attention_mask"]),
            "labels": torch.tensor(item[self.label_key]),
            "context_ids": context_ids
        }

train_dataset = TTPDataset(train_data)
val_dataset = TTPDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Model

In [18]:
from classModel import QACGBertForSequenceClassification
from transformers import BertConfig  # ← this is the correct one

max_context_id = 33
config = BertConfig.from_pretrained(
    "ibm-research/CTI-BERT",
    num_labels=len(labels), # This is allowed now
    num_context_ids=max_context_id + 1
)

# Add any other custom fields you need
config.context_dim = 68  # if required for your model
config.num_context_ids = 34 

model = QACGBertForSequenceClassification(config, num_labels=len(labels))
model.to(device)


/home/jupyter-st124945/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


QACGBertForSequenceClassification(
  (bert): ContextBertModel(
    (embeddings): BERTEmbeddings(
      (word_embeddings): Embedding(50000, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BERTLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ContextBERTEncoder(
      (context_layer): ModuleList(
        (0-11): 12 x Linear(in_features=1536, out_features=768, bias=True)
      )
      (layer): ModuleList(
        (0-11): 12 x ContextBERTLayer(
          (attention): ContextBERTAttention(
            (self): ContextBERTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (context_for_q): Linear(in_features=768, out_features=768, bias=T

# Training

In [51]:
# use this
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.optim import AdamW
from tqdm import tqdm
import time
import torch

def compute_metrics(preds, labels):
    # preds = preds.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Optimizer
# optimizer = AdamW(model.parameters(), lr=2e-5)
optimizer = AdamW(model.parameters(), lr=1e-6)

num_epochs = 5

print(f"{'Epoch':<6}{'Training Loss':<16}{'Validation Loss':<18}{'Accuracy':<10}{'Precision':<11}{'Recall':<9}{'F1'}")

for epoch in range(num_epochs):
    start_time = time.time()

    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        context_ids = batch["context_ids"].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            seq_lens=[input_ids.size(1)] * input_ids.size(0),
            context_ids=context_ids,
            labels=labels
        )

        loss, logits = outputs[:2]
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    epoch_train_loss = total_loss / len(train_loader)

    # Evaluation
    model.eval()
    all_preds, all_labels = [], []
    val_total_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            context_ids = batch["context_ids"].to(device)

            outputs = model(
                input_ids=input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                seq_lens=[input_ids.size(1)] * input_ids.size(0),
                context_ids=context_ids,
                labels=labels
            )

            loss, logits = outputs[:2]
            val_total_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss = val_total_loss / len(val_loader)
    metrics = compute_metrics(preds=np.array(all_preds), labels=np.array(all_labels))

    # print(f"{epoch+1:<6}{epoch_train_loss if epoch > 0 else 'No log':<16.6f}{val_loss:<18.6f}"
    #       f"{metrics['accuracy']:<10.6f}{metrics['precision']:<11.6f}"
    #       f"{metrics['recall']:<9.6f}{metrics['f1']:.6f}")
    train_loss_str = f"{epoch_train_loss:.6f}" if epoch > 0 else "No log"
    print(f"{epoch+1:<6}{train_loss_str:<16}{val_loss:<18.6f}"
          f"{metrics['accuracy']:<10.6f}{metrics['precision']:<11.6f}"
          f"{metrics['recall']:<9.6f}{metrics['f1']:.6f}")



Epoch Training Loss   Validation Loss   Accuracy  Precision  Recall   F1


1     No log          4.768497          0.102328  0.074183   0.102328 0.072063


2     2.398145        4.694597          0.148178  0.131304   0.148178 0.117844


3     1.706076        4.723936          0.168077  0.163138   0.168077 0.144451


4     1.323521        4.756011          0.175047  0.178341   0.175047 0.153007


5     1.081894        4.769297          0.190084  0.192082   0.190084 0.168840


In [ ]:
torch.save(model, 'model/full_model_edit.pth')

In [55]:
# model = torch.load('model/full_model.pth')
# model.to(device)
# model.eval

# Inference

In [56]:
from transformers import BertTokenizer
import torch
import torch.nn.functional as F
import spacy

# Load the spaCy NER model (make sure to download the model if not already installed)
nlp = spacy.load("en_core_web_sm")  # or any other model you are using


# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("ibm-research/CTI-BERT")
model = torch.load("model/full_model_edit.pth", map_location=device)
model.eval()

# Make sure `labels` and `context_map` are accessible
# If needed, regenerate from training data
labels = list(label_map.keys())
labels.sort(key=lambda x: label_map[x])  # Ensure label order matches model output

# Prediction function
def predict_single(text):
    # Run NER to extract context entities using spaCy
    doc = nlp(text)  # 'doc' is a spaCy document object
    aspects = list(set([ent.label_ for ent in doc.ents]))  # Extract the NER labels
    
    # Convert context to context_ids
    context_ids = [context_map.get(a, 0) for a in aspects]
    context_ids = torch.tensor([context_ids], dtype=torch.long).to(device)

    # Tokenize input text using the tokenizer
    encoded = tokenizer(text, padding="max_length", truncation=True, max_length=64, return_tensors="pt")
    input_ids = encoded['input_ids'].to(device)
    token_type_ids = encoded['token_type_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    # Sequence length
    seq_lens = [input_ids.size(1)]

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            seq_lens=seq_lens,
            context_ids=context_ids
        )
        logits = outputs
        probs = F.softmax(logits, dim=1).squeeze()
        pred_idx = torch.argmax(probs).item()
        pred_label = labels[pred_idx]

    return pred_label, probs.tolist()



/home/jupyter-st124945/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [65]:
# # Run the prediction
# input_text = input("Enter text to classify: ")
# predicted_label, prob_scores = predict_single(input_text)

# print(f"\nPredicted Label: {predicted_label}")
# print("Confidence Scores:")
# for i, label in enumerate(labels):
#     print(f"{label}: {prob_scores[i]:.4f}")


In [58]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
import spacy

# Load tokenizer and spaCy model
tokenizer = BertTokenizer.from_pretrained("ibm-research/CTI-BERT")
nlp = spacy.load("en_core_web_sm")

# Inverse label map
label_map_rev = {v: k for k, v in label_map.items()}


In [61]:
def predict_single(text, model, tokenizer, context_map, label_map_rev, device, max_len=64):
    # 1. Tokenize input
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=max_len, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    token_type_ids = inputs["token_type_ids"].to(device)

    seq_len = input_ids.size(1)

    # 2. Extract entities using spaCy for context
    doc = nlp(text)
    aspects = list(set([ent.text for ent in doc.ents if ent.text in context_map]))


    # 3. Encode context
    context_ids = [context_map.get(a, 0) for a in aspects]
    if not context_ids:
        context_ids = [0]  # default to padding if no valid entity
    context_ids = torch.tensor([context_ids], dtype=torch.long).to(device)

    # 4. Forward pass
    model.eval()
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            seq_lens=[seq_len],
            context_ids=context_ids,
        )

    logits = outputs
    probs = F.softmax(logits, dim=1).squeeze()
    pred_idx = torch.argmax(probs).item()
    pred_label = label_map_rev[pred_idx]

    return pred_label, probs.cpu().numpy()


In [66]:
import numpy as np

input_text = input("Enter text to classify: ")
predicted_label, confidence_scores = predict_single(input_text, model, tokenizer, context_map, label_map_rev, device)

print(f"\nPredicted Label: {predicted_label}")
print("\nTop 5 Confidence Scores:")

# Get indices of top 5 scores
top_indices = np.argsort(confidence_scores)[-5:][::-1]

for idx in top_indices:
    label = label_map_rev[idx]
    score = confidence_scores[idx]
    print(f"{label}: {score:.4f}")


Enter text to classify:  An Iranian state-sponsored actor has been observed scanning and attempting to abuse the Log4Shell flaw in publicly-exposed Java applications to deploy a hitherto undocumented PowerShell-based modular backdoor dubbed "CharmPower" for follow-on post-exploitation.  "The actor's attack setup was obviously rushed, as they used the basic open-source tool for the exploitation and based their operations on previous infrastructure, which made the attack easier to detect and attribute," researchers from Check Point said in a report published this week.  The Israeli cybersecurity company linked the attack to a group known as APT35, which is also tracked using the codenames Charming Kitten, Phosphorus, and TA453, citing overlaps with toolsets previously identified as infrastructure used by the threat actor.  Cybersecurity Log4Shell aka CVE-2021-44228 (CVSS score: 10.0) concerns a critical security vulnerability in the popular Log4j logging library that, if successfully exp


Predicted Label: T1098.003

Top 5 Confidence Scores:
T1098.003: 0.0329
T1569.001: 0.0271
T1027.008: 0.0257
T1218.011: 0.0231
T1129: 0.0180


In [81]:
from nltk.tokenize import sent_tokenize
import torch
import torch.nn.functional as F

def predict_single(text, model, tokenizer, context_map, label_map_rev, device, max_len=64):
    model.eval()

    # 1. Split the input text into sentences
    sentences = sent_tokenize(text)
    
    sentence_predictions = []

    # 2. Process each sentence separately
    for sentence in sentences:
        # 2.1 Tokenize input
        inputs = tokenizer(sentence, padding="max_length", truncation=True, max_length=max_len, return_tensors="pt")
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)
        token_type_ids = inputs["token_type_ids"].to(device)

        seq_len = input_ids.size(1)

        # 2.2 Extract entities using spaCy for context
        doc = nlp(sentence)
        aspects = list(set([ent.text for ent in doc.ents if ent.text in context_map]))

        # 2.3 Encode context
        context_ids = [context_map.get(a, 0) for a in aspects]
        if not context_ids:
            context_ids = [0]  # default to padding if no valid entity
        context_ids = torch.tensor([context_ids], dtype=torch.long).to(device)

        # 2.4 Forward pass through the model
        with torch.no_grad():
            outputs = model(
                input_ids=input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                seq_lens=[seq_len],
                context_ids=context_ids,
            )

        logits = outputs
        probs = F.softmax(logits, dim=1).squeeze()
        pred_idx = torch.argmax(probs).item()
        pred_label = label_map_rev[pred_idx]
        
        # Convert confidence scores to percentage
        probs_percent = probs.cpu().numpy() * 100  # Multiply by 100 to convert to percentage

        # Format the probabilities as percentages for clarity
        probs_percent_formatted = [f"{prob:.2f}%" for prob in probs_percent]

        sentence_predictions.append((sentence, pred_label, probs_percent_formatted))

    # Return the list of sentence predictions (each sentence, its label, and confidence scores as percentages)
    return sentence_predictions


In [83]:
# Sample input text
text = input("Enter text to classify: ")

# Call the predict_single function
sentence_predictions = predict_single(text, model, tokenizer, context_map, label_map_rev, device)

# Print the results
for sentence, pred_label, probs_percent in sentence_predictions:
    print(f"Sentence: {sentence}")
    print(f"Predicted Label: {pred_label}")
    # print(f"Confidence Scores: {probs_percent}")


Enter text to classify:   An Iranian state-sponsored actor has been observed scanning and attempting to abuse the Log4Shell flaw in publicly-exposed Java applications to deploy a hitherto undocumented PowerShell-based modular backdoor dubbed "CharmPower" for follow-on post-exploitation.  "The actor's attack setup was obviously rushed, as they used the basic open-source tool for the exploitation and based their operations on previous infrastructure, which made the attack easier to detect and attribute," researchers from Check Point said in a report published this week.  The Israeli cybersecurity company linked the attack to a group known as APT35, which is also tracked using the codenames Charming Kitten, Phosphorus, and TA453, citing overlaps with toolsets previously identified as infrastructure used by the threat actor.  Cybersecurity Log4Shell aka CVE-2021-44228 (CVSS score: 10.0) concerns a critical security vulnerability in the popular Log4j logging library that, if successfully ex

Sentence:  An Iranian state-sponsored actor has been observed scanning and attempting to abuse the Log4Shell flaw in publicly-exposed Java applications to deploy a hitherto undocumented PowerShell-based modular backdoor dubbed "CharmPower" for follow-on post-exploitation.
Predicted Label: T1585
Sentence: "The actor's attack setup was obviously rushed, as they used the basic open-source tool for the exploitation and based their operations on previous infrastructure, which made the attack easier to detect and attribute," researchers from Check Point said in a report published this week.
Predicted Label: T1546.007
Sentence: The Israeli cybersecurity company linked the attack to a group known as APT35, which is also tracked using the codenames Charming Kitten, Phosphorus, and TA453, citing overlaps with toolsets previously identified as infrastructure used by the threat actor.
Predicted Label: T1552.001
Sentence: Cybersecurity Log4Shell aka CVE-2021-44228 (CVSS score: 10.0) concerns a crit